In [1]:
import pandas as pd
import numpy as np
import re
import string  
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
from pathlib import Path

In [2]:
tweets=pd.read_excel("ExcelDocs\\SerbianTweetsDataSetCorrectedLabels.xlsx",index_col=0)

# Remove RT, #, @, numbers and new lines

In [3]:
def clean_text(text):
    text=re.sub(r'RT',' ',text)
    text=re.sub(r'(@[A-Za-z0-9!"#$%&\'()*+,-./:;<=>?@[\\\]^_`{|}~]+)',' ',text)
    text=re.sub(r'#',' ',text)
    text=re.sub(r'\w*\d\w*',' ',text)
    text=re.sub(r'(http|https)\S+', ' ', text)
    text=re.sub(r'\n',' ',text)
    
    return text

clean1=lambda x:clean_text(x)

In [4]:
clean_tweets=tweets
clean_tweets['Text']=clean_tweets['Text'].map(clean1)

# Emojis and emoticons to word conversion

In [5]:
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, (" _emo_"+"_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split())+" "))
    text=re.sub('_emo_Confusion','',text)
    return text


emoji=lambda x:convert_emojis(x)

In [6]:
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', (" _emo_"+"_".join(EMOTICONS[emot].replace(",","").split())+" "), text)
    text=re.sub('_emo_Confusion','',text)
    return text

emoticons=lambda x:convert_emoticons(x)

In [7]:
clean_tweets['Text']=clean_tweets['Text'].map(emoji)
clean_tweets['Text']=clean_tweets['Text'].map(emoticons)

# Remove unnecessary characters

In [8]:
#\u0107-ć \u010D-č \u0111-đ \u0161-š \u017E-ž 
def remove_unnecessary_characters(text):
    text=text.lower()
    text=re.sub(r"[^a-z\u0107\u010D\u0111\u0161\u017E .,!?_']",' ',text, re.UNICODE)
    return text

unnecessary_characters=lambda x:remove_unnecessary_characters(x)

In [9]:
def remove_successive_characters(text):
    text=re.sub(r'((.)\2{2,})',r'\2',text)
    text=re.sub(r"([.,!?'])", r' \1 ', text)
    return text

remove_successive=lambda x:remove_successive_characters(x)

In [10]:
clean_tweets['Text']=clean_tweets['Text'].map(unnecessary_characters)
clean_tweets['Text']=clean_tweets['Text'].map(remove_successive)

# Remove empty tweets

In [11]:
clean_tweets['Text'].replace([' ',''], np.nan, inplace=True)
clean_tweets.dropna(subset=['Text'], inplace=True)

In [12]:
clean_tweets.to_excel("ExcelDocs\\Processed_Tweets.xlsx")
clean_tweets.to_pickle("PickleDataframes\\Processed_Tweets.pkl")